In [1]:
import numpy as np
import pandas as pd
import re
from math import pi

from bokeh.io import output_file, show
from bokeh.palettes import Category20c, Plasma256, plasma
from bokeh.plotting import figure
from bokeh.transform import cumsum
from bokeh.models import LogColorMapper
from bokeh.palettes import Viridis6 as palette


In [2]:
# Load data
cleaned = pd.read_csv('investments_with_dummies.csv')
cleaned

,Unnamed: 0,company_permalink,company_name,company_category_list,company_country_code,company_state_code,company_region,company_city,investor_permalink,investor_name,...,News,Music,Financial Services,Android,Information Technology,Social Network Media,iPhone,Hospitality,Entertainment,Other
0,0,/organization/0-6-com,0-6.com,Curated Web,CHN,22,Beijing,Beijing,/organization/dt-capital-partners,DT Capital Partners,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,/organization/004-technologies,004 Technologies,Software,USA,IL,"Springfield, Illinois",Champaign,/organization/venturecapital-de,VCDE Venture Partners,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,/organization/01games-technology,01Games Technology,Games,HKG,NaN,Hong Kong,Hong Kong,/organization/cyberport-hong-kong,Cyberport Hong Kong,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,/organization/0xdata,H2O.ai,Analytics,USA,CA,SF Bay Area,Mountain View,/organization/capital-one,Capital One,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,/organization/0xdata,H2O.ai,Analytics,USA,CA,SF Bay Area,Mountain View,/organization/nexus-venture-partners,Nexus Venture Partners,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,5,/organization/0xdata,H2O.ai,Analytics,USA,CA,SF Bay Area,Mountain View,/organization/nexus-venture-partners,Nexus Venture Partners,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,6,/organization/0xdata,H2O.ai,Analytics,USA,CA,SF Bay Area,Mountain View,/organization/nexus-venture-partners,Nexus Venture Partners,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,7,/organization/0xdata,H2O.ai,Analytics,USA,CA,SF Bay Area,Mountain View,/organization/nexus-venture-partners,Nexus Venture Partners,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,8,/organization/0xdata,H2O.ai,Analytics,USA,CA,SF Bay Area,Mountain View,/organization/paxion-capital-partners,Paxion Capital Partners,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,9,/organization/0xdata,H2O.ai,Analytics,USA,CA,SF Bay Area,Mountain View,/organization/transamerica,Transamerica,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
# Beginning of Visualization for Q4

# Drop all investments that did not occur in the United States
stateInvestmentCount = cleaned.drop( cleaned[ cleaned['company_country_code'] != 'USA' ].index)
stateInvestmentCount = stateInvestmentCount.drop(['Unnamed: 0', 'company_permalink', 'company_country_code', 'company_region', 'company_city', 'investor_permalink', 'investor_country_code', 'investor_state_code', 'investor_region', 'investor_city', 'funding_round_permalink', 'funding_round_type', 'funding_round_code', 'funded_at'], axis=1)
stateInvestmentCount = stateInvestmentCount.groupby('company_state_code').count()
stateInvestmentCount.drop('VI', inplace=True)
stateInvestmentCount

,company_name,company_category_list,investor_name,raised_amount_usd,Software,Mobile,Biotechnology,E-Commerce,Enterprise Software,Curated Web,...,News,Music,Financial Services,Android,Information Technology,Social Network Media,iPhone,Hospitality,Entertainment,Other
company_state_code,,,,,,,,,,,,,,,,,,,,,
AK,4,4,4,3,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
AL,66,66,66,48,66,66,66,66,66,66,...,66,66,66,66,66,66,66,66,66,66
AR,90,90,90,75,90,90,90,90,90,90,...,90,90,90,90,90,90,90,90,90,90
AZ,477,477,477,427,477,477,477,477,477,477,...,477,477,477,477,477,477,477,477,477,477
CA,55620,55620,55619,50568,55620,55620,55620,55620,55620,55620,...,55620,55620,55620,55620,55620,55620,55620,55620,55620,55620
CO,1912,1912,1912,1754,1912,1912,1912,1912,1912,1912,...,1912,1912,1912,1912,1912,1912,1912,1912,1912,1912
CT,661,661,661,620,661,661,661,661,661,661,...,661,661,661,661,661,661,661,661,661,661
DC,623,623,623,548,623,623,623,623,623,623,...,623,623,623,623,623,623,623,623,623,623
DE,93,93,93,80,93,93,93,93,93,93,...,93,93,93,93,93,93,93,93,93,93


In [4]:
# Create bokeh figure

from bokeh.sampledata.us_states import data as states

output_file("investmentMap.html")

# Reverse color scheme so darker is greater
palette.reverse()

state_tags = stateInvestmentCount.index.values

# Remove Alaska and Hawaii for better map visualization
state_tags = np.delete(state_tags, [0, 11])
state_xs = [states[tag]["lons"] for tag in state_tags]
state_ys = [states[tag]["lats"] for tag in state_tags]
state_names = [states[tag]["name"] for tag in state_tags]

investment_counts = stateInvestmentCount.loc[(stateInvestmentCount.index != 'AK')&(stateInvestmentCount.index != 'HI'), 'company_name']
color_mapper = LogColorMapper(palette=palette)

data=dict(
    x=state_xs,
    y=state_ys,
    name=state_names,
    rate=investment_counts,
)

TOOLS = "pan,wheel_zoom,reset,hover,save"

p = figure(
    title="Distribution of Investments in US", plot_width = 1000, tools=TOOLS,
    x_axis_location=None, y_axis_location=None,
    tooltips=[
        ("Name", "@name"), ("Number of investments", "@rate"), ("(Long, Lat)", "($x, $y)")
    ])
p.grid.grid_line_color = None
p.hover.point_policy = "follow_mouse"

p.patches('x', 'y', source=data,
          fill_color={'field': 'rate', 'transform': color_mapper},
          fill_alpha=0.7, line_color="white", line_width=0.5)

show(p)

In [5]:
# Beginning of visualizations for Q1

# Get possible sector tags from dataframe
tags = cleaned.columns.values[-51:]
investmentSums = []

# For each tag, go through the dataframe of investments and add that investment to a running sum for that sector
# Then append that sum to a list of the investment sums for each of the sectors
for tag in tags:
    investmentSum = cleaned.loc[cleaned[tag] == 1, 'raised_amount_usd'].sum()
    investmentSums.append(investmentSum)

sectorInvestments = pd.Series({tag:total for (tag, total) in zip(tags, investmentSums)})
sectorInvestments = sectorInvestments.to_frame()
sectorInvestments = sectorInvestments.reset_index()
sectorInvestments = sectorInvestments.rename(columns={'index': 'Sectors', 0: "Total_Amount_Invested"})
sectorInvestments = sectorInvestments.sort_values(by=['Total_Amount_Invested'], ascending=False)
sectorInvestments


,Sectors,Total_Amount_Invested
50,Other,1.201234e+12
2,Biotechnology,3.004740e+11
0,Software,2.542332e+11
3,E-Commerce,2.012236e+11
1,Mobile,1.876358e+11
4,Enterprise Software,1.449233e+11
7,Health Care,1.378122e+11
12,Internet,1.118102e+11
18,Clean Technology,1.060021e+11
13,Technology,9.471280e+10


In [18]:
# Make bokeh figure of the sector breakdown of the top 20 sectors across all investments

output_file("aggregateSectorInvestmentPie.html")

data = sectorInvestments[:20].copy()
data['angle'] = data['Total_Amount_Invested']/data['Total_Amount_Invested'].sum() * 2*pi
data['color'] = Category20c[len(data)]

print(data)

p = figure(plot_height=600, title="Investment Distribution of All VCs", toolbar_location=None,
           tools="hover", tooltips="@Sectors: @Total_Amount_Invested", x_range=(-0.5, 1.0))

p.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_field='Sectors', source=data)

p.axis.axis_label=None
p.axis.visible=False
p.grid.grid_line_color = None

show(p)

                Sectors  Total_Amount_Invested     angle    color
50                Other           1.201234e+12  2.206482  #3182bd
2         Biotechnology           3.004740e+11  0.551924  #6baed6
0              Software           2.542332e+11  0.466987  #9ecae1
3            E-Commerce           2.012236e+11  0.369617  #c6dbef
1                Mobile           1.876358e+11  0.344658  #e6550d
4   Enterprise Software           1.449233e+11  0.266202  #fd8d3c
7           Health Care           1.378122e+11  0.253140  #fdae6b
12             Internet           1.118102e+11  0.205378  #fdd0a2
18     Clean Technology           1.060021e+11  0.194710  #31a354
13           Technology           9.471280e+10  0.173973  #74c476
6           Advertising           8.465768e+10  0.155503  #a1d99b
11              Finance           8.389714e+10  0.154106  #c7e9c0
24       Semiconductors           7.666543e+10  0.140823  #756bb1
9             Analytics           7.307581e+10  0.134229  #9e9ac8
8         

In [7]:
# Sort investors by amount invested
ventureInvestments = cleaned.groupby(['investor_name']).sum()
ventureInvestments = ventureInvestments.drop(columns=['Unnamed: 0'])
topFifty = list(ventureInvestments.sort_values(by=['raised_amount_usd']).tail(50).index)
topFifty
ventureInvestments.sort_values(by=['raised_amount_usd'])

,raised_amount_usd,Software,Mobile,Biotechnology,E-Commerce,Enterprise Software,Curated Web,Advertising,Health Care,Social Media,...,News,Music,Financial Services,Android,Information Technology,Social Network Media,iPhone,Hospitality,Entertainment,Other
investor_name,,,,,,,,,,,,,,,,,,,,,
The Florida Technology Seed Capital Fund,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
Dato Dr Maimunah,0.000000e+00,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Dato Sharil Tarmizi,0.000000e+00,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Dato Syed Budriz Putra,0.000000e+00,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bharat Banka,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Datran Media,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
Datuk Yvonne Chia,0.000000e+00,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Leonora Valvo,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
Paddy MccGwire,0.000000e+00,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [8]:
# Isolate investments from top 50 VCs
mask = []

for row in cleaned['investor_name']:
    if row in topFifty:
        mask.append(True)
    else:
        mask.append(False)

topFiftyInvestments = cleaned[mask]
topFiftyCleanedInvestments = topFiftyInvestments.drop(['Unnamed: 0', 'company_permalink', 'company_country_code', 'company_state_code', 'company_region', 'company_city', 'investor_permalink', 'investor_country_code', 'investor_state_code', 'investor_region', 'investor_city', 'funding_round_permalink', 'funding_round_type', 'funding_round_code', 'funded_at'], axis=1)
topFiftyCleanedInvestments

,company_name,company_category_list,investor_name,raised_amount_usd,Software,Mobile,Biotechnology,E-Commerce,Enterprise Software,Curated Web,...,News,Music,Financial Services,Android,Information Technology,Social Network Media,iPhone,Hospitality,Entertainment,Other
20,One Inc.,Mobile,General Catalyst Partners,1000050.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
44,1000memories,Curated Web,Greylock Partners,2520000.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
91,1006.tv,Games|Media,Sequoia Capital,10000000.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
104,100Plus,Analytics,Founders Fund,500000.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
105,100Plus,Analytics,Greylock Partners,750000.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
106,1010data,Software,Norwest Venture Partners - NVP,35000000.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
126,10X Genomics,Biotechnology|Technology,Venrock,55000000.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
148,123Greetings,Internet,Intel Capital,5400000.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
181,140 Proof,Advertising|Big Data Analytics|Interest Graph,Founders Fund,3000000.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
182,140 Proof,Advertising|Big Data Analytics|Interest Graph,Founders Fund,2500000.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [9]:
# Get possible sector tags from dataframe
tags = cleaned.columns.values[-51:]
investmentSums = []

# For each tag, go through the dataframe of investments and add that investment to a running sum for that sector
# Then append that sum to a list of the investment sums for each of the sectors
for tag in tags:
    investmentSum = topFiftyCleanedInvestments.loc[topFiftyCleanedInvestments[tag] == 1, 'raised_amount_usd'].sum()
    investmentSums.append(investmentSum)

topFiftySectorInvestments = pd.Series({tag:total for (tag, total) in zip(tags, investmentSums)})
topFiftySectorInvestments = topFiftySectorInvestments.to_frame()
topFiftySectorInvestments = topFiftySectorInvestments.reset_index()
topFiftySectorInvestments = topFiftySectorInvestments.rename(columns={'index': 'Sectors', 0: "Total_Amount_Invested"})
topFiftySectorInvestments = topFiftySectorInvestments.sort_values(by=['Total_Amount_Invested'], ascending=False)
topFiftySectorInvestments

,Sectors,Total_Amount_Invested
50,Other,3.087448e+11
3,E-Commerce,5.781430e+10
0,Software,5.644804e+10
1,Mobile,5.156748e+10
12,Internet,4.386083e+10
4,Enterprise Software,3.898545e+10
13,Technology,2.843817e+10
24,Semiconductors,2.783208e+10
8,Social Media,2.608196e+10
6,Advertising,2.338961e+10


In [10]:
# Make bokeh figure of the sector breakdown of the top 20 sectors across all investments from the top 50 VCs

output_file("topFiftySectorPie.html")

data = topFiftySectorInvestments[:20].copy()
data['angle'] = data['Total_Amount_Invested']/data['Total_Amount_Invested'].sum() * 2*pi
data['color'] = Category20c[len(data)]

print(data)

p = figure(plot_height=600, title="Investment Distribution of Top 50 VCs", toolbar_location=None,
           tools="hover", tooltips="@Sectors: @Total_Amount_Invested", x_range=(-0.5, 1.0))

p.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_field='Sectors', source=data)

p.axis.axis_label=None
p.axis.visible=False
p.grid.grid_line_color = None

show(p)

                Sectors  Total_Amount_Invested     angle    color
50                Other           3.087448e+11  2.329531  #3182bd
3            E-Commerce           5.781430e+10  0.436218  #6baed6
0              Software           5.644804e+10  0.425910  #9ecae1
1                Mobile           5.156748e+10  0.389085  #c6dbef
12             Internet           4.386083e+10  0.330937  #e6550d
4   Enterprise Software           3.898545e+10  0.294152  #fd8d3c
13           Technology           2.843817e+10  0.214571  #fdae6b
24       Semiconductors           2.783208e+10  0.209998  #fdd0a2
8          Social Media           2.608196e+10  0.196793  #31a354
6           Advertising           2.338961e+10  0.176479  #74c476
32               Travel           2.236253e+10  0.168729  #a1d99b
2         Biotechnology           2.042152e+10  0.154084  #c7e9c0
11              Finance           1.967413e+10  0.148445  #756bb1
9             Analytics           1.733629e+10  0.130805  #9e9ac8
18     Cle

In [11]:
# Beginning of Visualizations for Q5

# Change all of the dates that the investments occured on to just the year
def dateToYear(s):
    return s[0:4]

cleanedYears = cleaned.copy()
cleanedYears.loc[:, 'funded_at'] = cleanedYears['funded_at'].map(dateToYear)

cleanedYears = cleanedYears[(cleanedYears['funded_at'] != '1977') & (cleanedYears['funded_at'] != '1979') & (cleanedYears['funded_at'] != '1982')& (cleanedYears['funded_at'] != '1984')]
years = sorted(cleanedYears['funded_at'].unique())
years

['1985',
 '1986',
 '1987',
 '1988',
 '1989',
 '1990',
 '1991',
 '1992',
 '1993',
 '1994',
 '1995',
 '1996',
 '1997',
 '1998',
 '1999',
 '2000',
 '2001',
 '2002',
 '2003',
 '2004',
 '2005',
 '2006',
 '2007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015']

In [12]:
tags = cleanedYears.columns.values[-51:]
investmentSums = []
yearlyInvestments = pd.DataFrame({tag: [] for tag in sorted(tags)})

for year in years[10:]:
    print(year)
    for tag in tags:
        investmentSum = cleanedYears.loc[(cleanedYears[tag] == 1) & (cleanedYears['funded_at'].values == year), 'raised_amount_usd'].sum()
        investmentSums.append(investmentSum)
    
    investmentPercentages = investmentSums
    
    sectorInvestmentsForYear = pd.Series({tag:total for (tag, total) in zip(tags, investmentPercentages)})
    sectorInvestmentsForYear = sectorInvestmentsForYear.to_frame()
    sectorInvestmentsForYear = sectorInvestmentsForYear.reset_index()
    sectorInvestmentsForYear = sectorInvestmentsForYear.rename(columns={'index': 'Sectors', 0: "Total_Amount_Invested"})
    sectorInvestmentsForYear = sectorInvestmentsForYear.sort_values(by=['Sectors'])
    print(list(sectorInvestmentsForYear['Total_Amount_Invested']))
    yearlyInvestment = pd.DataFrame({sector:[investment] for (sector, investment) in zip(sorted(tags), list(sectorInvestmentsForYear['Total_Amount_Invested']))})
    print(yearlyInvestment)
    yearlyInvestments = yearlyInvestments.append(yearlyInvestment)
    investmentSums = []

yearlyInvestments[['Other', 'Semiconductors']]


1995
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8000000.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 24400000.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 24400000.0, 8000000.0, 0.0, 0.0, 16400000.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8000000.0, 0.0, 16400000.0, 0.0, 0.0, 2000000.0, 0.0]
   Advertising  Analytics  Android  Apps  Big Data  Biotechnology  \
0          0.0        0.0      0.0   0.0       0.0            0.0   

   Clean Technology  Cloud Computing  Curated Web  Digital Media  ...  \
0               0.0              0.0          0.0            0.0  ...   

   Services  Social Media  Social Network Media   Software  Startups  \
0       0.0           0.0                   0.0  8000000.0       0.0   

   Technology  Travel  Video  Web Hosting  iPhone  
0  16400000.0     0.0    0.0    2000000.0     0.0  

[1 rows x 51 columns]
1996
[2000000.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 42000000.0, 3100000.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3930000.0, 0.0, 0.0, 0.0

[9551150000.0, 390230000.0, 500000.0, 303700000.0, 100000000.0, 8198870182.0, 22087939.0, 77780000.0, 385452870.0, 79600000.0, 442872572.0, 28400000.0, 1325094048.0, 75300000.0, 26400000.0, 0.0, 244102162.0, 294860000.0, 230075634.0, 1247692062.0, 4130059281.0, 957125000.0, 19000000.0, 236971859.0, 636100000.0, 1723978028.0, 1000000.0, 298806878.0, 3431530653.0, 163700000.0, 1499720000.0, 14100000.0, 1287615000.0, 0.0, 30933414245.0, 569208517.0, 76490000.0, 762400000.0, 17750000.0, 995500000.0, 3018180304.0, 2265495292.0, 113406878.0, 0.0, 5226750464.0, 0.0, 2577127642.0, 39600000.0, 429324000.0, 2235570000.0, 60900000.0]
    Advertising    Analytics   Android         Apps     Big Data  \
0  9.551150e+09  390230000.0  500000.0  303700000.0  100000000.0   

   Biotechnology  Clean Technology  Cloud Computing  Curated Web  \
0   8.198870e+09        22087939.0       77780000.0  385452870.0   

   Digital Media  ...      Services  Social Media  Social Network Media  \
0     79600000.0  ..

[2882267816.0, 1600039520.5604289, 165017869.0, 586609145.0, 347588102.0, 22825073128.0, 8399119504.0, 851121406.372719, 1579541260.3004968, 537168199.0, 2186406083.0, 671272190.0, 3878453369.0, 498432262.0, 367568559.0, 1538558825.0, 2233705420.300497, 191418104.0, 1961553671.0, 2922619539.7188463, 10697309747.0, 1179757986.0, 41801875.0, 199077271.3727191, 14778055643.0, 1322275016.0, 269835055.0, 782046977.0, 1467760689.0, 1437556148.0, 17861945600.0, 1984773602.0, 539435025.0, 1994381854.0, 27855739351.760227, 337105617.0, 1199905797.0, 459062661.0, 416556628.0, 1459661515.0, 3334089545.0, 120639915.0, 1678327999.0, 203914105.0, 7921416648.0, 39747850.0, 2865063421.0, 388362529.0, 759212413.0, 2908158272.0, 298978881.0]
    Advertising     Analytics      Android         Apps     Big Data  \
0  2.882268e+09  1.600040e+09  165017869.0  586609145.0  347588102.0   

   Biotechnology  Clean Technology  Cloud Computing   Curated Web  \
0   2.282507e+10      8.399120e+09     8.511214e+08 

[9217927877.349434, 18808647639.18239, 2850574623.888272, 15718199589.927528, 11632587232.489902, 48271810280.0636, 7060168614.834522, 14914523209.986744, 13145390003.246742, 2986745803.8183002, 59224175291.529236, 17900787830.67588, 37214521372.30233, 11431347167.754864, 6513532396.340408, 16451942744.98979, 28299254678.281845, 19603413741.795914, 5505467561.059725, 7243250932.526068, 21792985732.8051, 11792627896.133253, 8197556619.112007, 5132338433.818816, 31166727788.176277, 11999851187.587873, 9503194444.88863, 2045150570.1506085, 8366428512.669865, 2205151401.283682, 30319192655.46849, 8223612059.927635, 3541781780.797776, 2907274819.8482203, 353481380768.1964, 11170731637.875717, 19063628772.54905, 4079047361.5944214, 2894471590.1076965, 11218463103.647038, 1731363019.682397, 12043068306.079288, 10425203425.729067, 817298660.6871144, 51943723937.14671, 3873404417.9363585, 21337901897.569695, 28261250994.790535, 4243549286.082173, 3432088185.833732, 1473607194.6708534]
    Adver

,Other,Semiconductors
0,2.440000e+07,0.000000e+00
0,7.100000e+06,0.000000e+00
0,8.360000e+07,9.900000e+06
0,3.211040e+08,9.000000e+06
0,4.780731e+09,1.950000e+07
0,1.576779e+10,6.407000e+08
0,2.194759e+10,1.630000e+09
0,1.533889e+10,2.295009e+09
0,3.093341e+10,3.018180e+09
0,2.402204e+10,3.193605e+09


In [13]:
yearlyInvestments.index = years[10:]
yearlyInvestments.loc[:,:] = yearlyInvestments.loc[:,:].div(yearlyInvestments.sum(axis=1), axis=0)
yearlyInvestments = yearlyInvestments * 100
yearlyInvestments

,Advertising,Analytics,Android,Apps,Big Data,Biotechnology,Clean Technology,Cloud Computing,Curated Web,Digital Media,...,Services,Social Media,Social Network Media,Software,Startups,Technology,Travel,Video,Web Hosting,iPhone
1995,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,7.434944,0.000000,15.241636,0.000000,0.000000,1.858736,0.000000
1996,1.570722,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,32.985157,2.434619,...,1.570722,1.256577,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.282887,0.000000
1997,2.996105,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,14.106661,0.000000,...,2.996105,0.000000,0.000000,10.736043,0.000000,0.000000,0.000000,0.000000,2.496754,0.000000
1998,1.065583,0.000000,0.000000,9.412651,0.000000,0.568311,0.000000,0.000000,5.860707,0.000000,...,0.000000,0.355194,0.000000,0.887986,0.000000,0.000000,0.000000,0.000000,5.919906,0.000000
1999,1.431877,0.044919,0.000000,0.002983,0.000000,0.270916,0.000000,0.477292,4.755132,0.859126,...,7.135520,0.387800,0.000000,11.051262,0.000000,1.617799,0.016156,0.229518,9.692614,0.000000
2000,1.390537,0.017385,0.102778,1.912413,0.000000,1.266142,0.660719,0.358791,2.325069,0.001835,...,13.393063,0.000000,0.361655,9.998855,0.000000,6.245746,0.012572,0.073413,0.584552,0.009177
2001,0.714918,0.424854,0.058544,0.069603,0.000000,2.171440,0.100827,0.067243,1.172678,1.230760,...,4.475399,0.003252,0.153772,10.679034,0.361024,5.272092,0.047649,0.084564,5.280386,0.000000
2002,0.288719,0.492345,0.045687,0.047872,1.676500,7.410608,0.385754,1.605984,1.111078,0.150964,...,5.129978,0.057803,0.246310,8.408842,0.000000,4.052412,0.099319,1.925790,1.781352,0.009932
2003,11.010607,0.449859,0.000576,0.350107,0.115280,9.451693,0.025463,0.089665,0.444352,0.091763,...,2.611673,0.130736,0.000000,6.025420,0.000000,2.970924,0.045651,0.494927,2.577175,0.070206
2004,0.387829,0.206826,0.130883,0.214835,0.000000,16.772073,0.559932,0.182448,0.928015,0.036501,...,4.082640,0.064524,0.143627,7.056106,0.083452,2.760259,0.538316,0.481022,2.106687,0.104289


In [14]:
# Making Bokeh figure of Percentage Bar Graph of the top 50 Sectors in terms of investment from 1995 to 2015

from bokeh.core.properties import value
from bokeh.io import show, output_file
from bokeh.plotting import figure

output_file("percentageBarGraphTop50.html")

colors = Category20c[20]*3
colors = colors[:51]

data = yearlyInvestments

p = figure(x_range=data.index.values, plot_height=1000, plot_width=1500, title="Investments by Sector over the Years",
           toolbar_location=None, tools="hover", tooltips="$name: @$name %")

p.vbar_stack(tags, x='index', width=0.9, color=colors, source=data)

p.y_range.start = 0
p.x_range.range_padding = 0.1
p.xgrid.grid_line_color = None
p.axis.minor_tick_line_color = None
p.outline_line_color = None

show(p)

In [15]:
tags = cleanedYears.drop(columns = ['Other']).columns.values[-50:]
investmentSums = []
yearlyInvestments = pd.DataFrame({tag: [] for tag in sorted(tags)})

for year in years[10:]:
    print(year)
    for tag in tags:
        investmentSum = cleanedYears.loc[(cleanedYears[tag] == 1) & (cleanedYears['funded_at'].values == year), 'raised_amount_usd'].sum()
        investmentSums.append(investmentSum)
    
    investmentPercentages = investmentSums    
        
    sectorInvestmentsForYear = pd.Series({tag:total for (tag, total) in zip(tags, investmentPercentages)})
    sectorInvestmentsForYear = sectorInvestmentsForYear.to_frame()
    sectorInvestmentsForYear = sectorInvestmentsForYear.reset_index()
    sectorInvestmentsForYear = sectorInvestmentsForYear.rename(columns={'index': 'Sectors', 0: "Total_Amount_Invested"})
    sectorInvestmentsForYear = sectorInvestmentsForYear.sort_values(by=['Sectors'])
    print(list(sectorInvestmentsForYear['Total_Amount_Invested']))
    yearlyInvestment = pd.DataFrame({sector:[investment] for (sector, investment) in zip(sorted(tags), list(sectorInvestmentsForYear['Total_Amount_Invested']))})
    print(yearlyInvestment)
    yearlyInvestments = yearlyInvestments.append(yearlyInvestment)
    investmentSums = []

yearlyInvestments



yearlyInvestments.index = years[10:]

yearlyInvestments.loc[:,:] = yearlyInvestments.loc[:,:].div(yearlyInvestments.sum(axis=1), axis=0)
yearlyInvestments = yearlyInvestments * 100
yearlyInvestments


1995
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8000000.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 24400000.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8000000.0, 0.0, 0.0, 16400000.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8000000.0, 0.0, 16400000.0, 0.0, 0.0, 2000000.0, 0.0]
   Advertising  Analytics  Android  Apps  Big Data  Biotechnology  \
0          0.0        0.0      0.0   0.0       0.0            0.0   

   Clean Technology  Cloud Computing  Curated Web  Digital Media  ...  \
0               0.0              0.0          0.0            0.0  ...   

   Services  Social Media  Social Network Media   Software  Startups  \
0       0.0           0.0                   0.0  8000000.0       0.0   

   Technology  Travel  Video  Web Hosting  iPhone  
0  16400000.0     0.0    0.0    2000000.0     0.0  

[1 rows x 50 columns]
1996
[2000000.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 42000000.0, 3100000.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3930000.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

[9551150000.0, 390230000.0, 500000.0, 303700000.0, 100000000.0, 8198870182.0, 22087939.0, 77780000.0, 385452870.0, 79600000.0, 442872572.0, 28400000.0, 1325094048.0, 75300000.0, 26400000.0, 0.0, 244102162.0, 294860000.0, 230075634.0, 1247692062.0, 4130059281.0, 957125000.0, 19000000.0, 236971859.0, 636100000.0, 1723978028.0, 1000000.0, 298806878.0, 3431530653.0, 163700000.0, 1499720000.0, 14100000.0, 1287615000.0, 0.0, 569208517.0, 76490000.0, 762400000.0, 17750000.0, 995500000.0, 3018180304.0, 2265495292.0, 113406878.0, 0.0, 5226750464.0, 0.0, 2577127642.0, 39600000.0, 429324000.0, 2235570000.0, 60900000.0]
    Advertising    Analytics   Android         Apps     Big Data  \
0  9.551150e+09  390230000.0  500000.0  303700000.0  100000000.0   

   Biotechnology  Clean Technology  Cloud Computing  Curated Web  \
0   8.198870e+09        22087939.0       77780000.0  385452870.0   

   Digital Media  ...      Services  Social Media  Social Network Media  \
0     79600000.0  ...  2.265495e+09

[2882267816.0, 1600039520.5604289, 165017869.0, 586609145.0, 347588102.0, 22825073128.0, 8399119504.0, 851121406.372719, 1579541260.3004968, 537168199.0, 2186406083.0, 671272190.0, 3878453369.0, 498432262.0, 367568559.0, 1538558825.0, 2233705420.300497, 191418104.0, 1961553671.0, 2922619539.7188463, 10697309747.0, 1179757986.0, 41801875.0, 199077271.3727191, 14778055643.0, 1322275016.0, 269835055.0, 782046977.0, 1467760689.0, 1437556148.0, 17861945600.0, 1984773602.0, 539435025.0, 1994381854.0, 337105617.0, 1199905797.0, 459062661.0, 416556628.0, 1459661515.0, 3334089545.0, 120639915.0, 1678327999.0, 203914105.0, 7921416648.0, 39747850.0, 2865063421.0, 388362529.0, 759212413.0, 2908158272.0, 298978881.0]
    Advertising     Analytics      Android         Apps     Big Data  \
0  2.882268e+09  1.600040e+09  165017869.0  586609145.0  347588102.0   

   Biotechnology  Clean Technology  Cloud Computing   Curated Web  \
0   2.282507e+10      8.399120e+09     8.511214e+08  1.579541e+09   

  

[9217927877.349434, 18808647639.18239, 2850574623.888272, 15718199589.927528, 11632587232.489902, 48271810280.0636, 7060168614.834522, 14914523209.986744, 13145390003.246742, 2986745803.8183002, 59224175291.529236, 17900787830.67588, 37214521372.30233, 11431347167.754864, 6513532396.340408, 16451942744.98979, 28299254678.281845, 19603413741.795914, 5505467561.059725, 7243250932.526068, 21792985732.8051, 11792627896.133253, 8197556619.112007, 5132338433.818816, 31166727788.176277, 11999851187.587873, 9503194444.88863, 2045150570.1506085, 8366428512.669865, 2205151401.283682, 30319192655.46849, 8223612059.927635, 3541781780.797776, 2907274819.8482203, 11170731637.875717, 19063628772.54905, 4079047361.5944214, 2894471590.1076965, 11218463103.647038, 1731363019.682397, 12043068306.079288, 10425203425.729067, 817298660.6871144, 51943723937.14671, 3873404417.9363585, 21337901897.569695, 28261250994.790535, 4243549286.082173, 3432088185.833732, 1473607194.6708534]
    Advertising     Analytic

,Advertising,Analytics,Android,Apps,Big Data,Biotechnology,Clean Technology,Cloud Computing,Curated Web,Digital Media,...,Services,Social Media,Social Network Media,Software,Startups,Technology,Travel,Video,Web Hosting,iPhone
1995,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,9.615385,0.000000,19.711538,0.000000,0.000000,2.403846,0.000000
1996,1.663478,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,34.933045,2.578391,...,1.663478,1.330783,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.653913,0.000000
1997,3.786445,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,17.827843,0.000000,...,3.786445,0.000000,0.000000,13.568093,0.000000,0.000000,0.000000,0.000000,3.155370,0.000000
1998,1.719185,0.000000,0.000000,15.186130,0.000000,0.916898,0.000000,0.000000,9.455515,0.000000,...,0.000000,0.573062,0.000000,1.432654,0.000000,0.000000,0.000000,0.000000,9.551025,0.000000
1999,2.003258,0.062844,0.000000,0.004173,0.000000,0.379023,0.000000,0.667753,6.652635,1.201955,...,9.982901,0.542549,0.000000,15.461193,0.000000,2.263371,0.022603,0.321106,13.560385,0.000000
2000,1.956825,0.024465,0.144634,2.691231,0.000000,1.781770,0.929792,0.504906,3.271938,0.002583,...,18.847302,0.000000,0.508937,14.070824,0.000000,8.789286,0.017692,0.103310,0.822608,0.012914
2001,1.111709,0.660654,0.091038,0.108234,0.000000,3.376621,0.156787,0.104564,1.823532,1.913850,...,6.959313,0.005058,0.239118,16.606058,0.561398,8.198182,0.074094,0.131499,8.211079,0.000000
2002,0.415237,0.708091,0.065707,0.068849,2.411147,10.657957,0.554792,2.309730,1.597956,0.217117,...,7.377949,0.083133,0.354244,12.093619,0.000000,5.828190,0.142840,2.769676,2.561945,0.014284
2003,17.113206,0.699192,0.000896,0.544152,0.179174,14.690268,0.039576,0.139362,0.690632,0.142623,...,4.059185,0.203196,0.000000,9.364993,0.000000,4.617550,0.070953,0.769238,4.005567,0.109117
2004,0.517467,0.275960,0.174632,0.286647,0.000000,22.378374,0.747097,0.243434,1.238216,0.048702,...,5.447319,0.086092,0.191636,9.414709,0.111347,3.682914,0.718256,0.641810,2.810877,0.139149


In [16]:
# Making Bokeh figure of Percentage Bar Graph of the top 50 Sectors (without the other category) in terms of investment from 1995 to 2015

from bokeh.core.properties import value
from bokeh.io import show, output_file
from bokeh.plotting import figure

output_file("percentageBarGraphTop50WithoutOther.html")

colors = Category20c[20]*3
colors = colors[:50]

data = yearlyInvestments

p = figure(x_range=data.index.values, plot_height=1000, plot_width=1500, title="Investments by Sector over the Years Without Other",
           toolbar_location=None, tools="hover", tooltips="$name: @$name %")

p.vbar_stack(tags, x='index', width=0.9, color=colors, source=data)

p.y_range.start = 0
p.x_range.range_padding = 0.1
p.xgrid.grid_line_color = None
p.axis.minor_tick_line_color = None
p.outline_line_color = None

show(p)
